### jupyter magic commands
Jupyter provides a number of magic commands, allowing you to integrate external functions and methods that are not natively found in RevBayes into your RevBayes workflow.

To list the magic commands available in Jupyter:

In [1]:
%lsmagic

Available line magics:
%activity  %cd  %connect_info  %conversation  %dot  %download  %edit  %get  %help  %html  %include  %install  %install_magic  %javascript  %jigsaw  %kernel  %kx  %latex  %load  %ls  %lsmagic  %macro  %magic  %matplotlib  %parallel  %plot  %pmap  %px  %python  %reload_magics  %restart  %run  %scheme  %set  %shell

Available cell magics:
%%activity  %%brain  %%conversation  %%debug  %%dot  %%file  %%help  %%html  %%javascript  %%kx  %%latex  %%macro  %%pipe  %%processing  %%px  %%python  %%scheme  %%shell  %%show  %%time  %%tutor


### Simulate a tree in `RevBayes`
Suppose we simulate and save a `Tree` object to file using `writeNexus` in RevBayes. We can then open that tree file from within the same Juptyer notebook using the `%python` magic command.

First, create and save the `Tree` object

In [2]:
n_taxa = 10
for (i in 1:n_taxa) {
    taxa[i] = taxon("T"+i, 0.0)
}
phy ~ dnBDP(lambda=1, rootAge=1, taxa=taxa)
writeNexus(data=phy, filename="example/output/magic_test.tre")

### Open output in `tracer`
Let's view the tree in [FigTree](http://tree.bio.ed.ac.uk/software/figtree/). Launch terminal commands using the magic function `%shell`

In [3]:
%shell figtree example/output/magic_test.tre

Of course, this example relies on `figtree` existing in the user's environment `PATH` variable.

### Executing `python` code
Use `%%python` to indicate the next cell conatins python commands

In [4]:
%%python

# python packages
import dendropy
from Bio import Phylo

# the tree's filename
phy_fn = 'example/output/magic_test.tre'

# visualize a tree in Dendropy
dtree = dendropy.Tree.get(path=phy_fn,schema='nexus')
print('Dendropy\n'+dtree.as_ascii_plot())

# visualize a tree in Biopython
bptree = Phylo.read(phy_fn, 'nexus')
Phylo.draw(bptree)

Dendropy
                                                                                               /----------------------- T1 
                                                                       /-----------------------+                           
                                                                       |                       \----------------------- T10
                                               /-----------------------+                                                   
                                               |                       |                       /----------------------- T3 
/----------------------------------------------+                       \-----------------------+                           
|                                              |                                               \----------------------- T9 
|                                              |                                                                           

This draws the tree in a pyplot window.
Full-featured kernels enable inline figure presentation through the magic command "`%matplotlib inline`".
Unfortunately, this is not currently supported in metakernel ([link](https://groups.google.com/forum/#!msg/jupyter/A6uw-Pp5fE0/udnhbhS_AgAJ)).
I've made no progress so far on a workaround.

### Executing `R` code through `rpy2`
`R` does not natively support magic commands.
However, we can the package `rpy2` to execute `R` commands through the `%%python` environment.

In [5]:
%%python

# python packages
from rpy2.robjects import r
import dendropy

# read tree into dendropy
phy_fn = 'example/output/magic_test.tre'
dtree = dendropy.Tree.get(path=phy_fn,schema='nexus')

# extract Newick string from file
phy_str = dtree.as_string("newick")

# parse Newick string in R
rtree = r("""
library('ape')
phy_r=read.tree(text='%s')
""" % phy_str)

# read the R object from the global rpy2 environment
rtree=r['phy_r']
print("ape tree as rpy2 object:")
print(rtree)

# store R values into python
tip_labels = rtree[ rtree.names.index('tip.label') ]
print("Tip labels as rpy2 object:")
print(tip_labels)

ape tree as rpy2 object:

Phylogenetic tree with 10 tips and 9 internal nodes.

Tip labels:
	T1, T10, T3, T9, T6, T5, ...

Rooted; includes branch lengths.

Tip labels as rpy2 object:
 [1] "T1"  "T10" "T3"  "T9"  "T6"  "T5"  "T8"  "T7"  "T4"  "T2" 



### Importing `python` objects into `RevBayes`

This feature is slated for future development.